# State-of-the-art Implementation 
In meiner Recherche bin ich den Blog: "CartPole Balance using Python | OpenAI Gym | Reinforcement Learning Project Tutorial" gestoßen. UNd würde es im Rahmen des Projektes ausprobieren. Aus dem einfachen Grund, dass meine Implementation mit Q-Net beziehungsweise Dueling Q-Net als Optimierung keine befriegigende Leistung geliefert hat. Es ist sehr Interessant zu sehen, wie bereits existierende Lösungen fnktionieren. Ich würde diesen Ansatz als Alternative ansehen und versuchen zu verstehen was sein Gedankengang war.

Code von<br> 
Aswintechguy, <br>
Titel: CartPole Balance using Python | OpenAI Gym | Reinforcement Learning Project Tutorial <br>
URL: https://www.hackersrealm.net/post/cartpole-balance-reinforcement-learning <br>
Zugriffsdatum: 20.07.2023

In [22]:
# install modules
# !pip install gym stable_baselinese

import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy




Im nächsten Schritt folgt eine einfache Trainingsschleife, in der der Agent zehn Episoden lang trainiert wird. Für jede Episode wird die Umgebung zurückgesetzt, und der Agent interagiert mit der Umgebung, bis die Episode abgeschlossen ist. Der Agent wählt bei jedem Schritt zufällige Aktionen (env.action_space.sample()), und die Umgebung gibt den nächsten Zustand (n_state), die Belohnung der Aktion und eine Flagge, die angibt, ob die Episode beendet ist (done), zurück. Die gesamte Belohnung der Episode wird in der Variable score gesammelt.

In [26]:
env_name = 'CartPole-v1'
env = gym.make(env_name)


for episode in range(1, 11):
    score = 0
    state = env.reset()
    done = False
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        
    print('Episode:', episode, 'Score:', score)
env.close()



Episode: 1 Score: 13.0
Episode: 2 Score: 16.0
Episode: 3 Score: 22.0
Episode: 4 Score: 10.0
Episode: 5 Score: 42.0
Episode: 6 Score: 14.0
Episode: 7 Score: 54.0
Episode: 8 Score: 22.0
Episode: 9 Score: 17.0
Episode: 10 Score: 20.0


In [28]:
# Hier wird eine neue Instanz der Umgebung erstellt und in eine DummyVecEnv gehüllt. 
# Dies ist hilfreich, um mehrere Umgebungen parallel zu trainieren. 
# In diesem Code wird jedoch nur eine Umgebung verwendet.
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])


Im nächsten Schritt wird ein PPO Modell definiert.<br>
Proximal Policy Optimization, oder PPO, ist ein on-policy Reinforcement-Learning-Algorithmus, der aufgrund seiner Effektivität und Stabilität beliebt geworden ist. Das Ziel besteht darin, die optimale Richtlinie zu finden, indem die Richtlinienparameter iterativ aktualisiert werden. Er gehört zur Familie der Policy-Gradient-Methoden und verwendet eine Ersatzzielfunktion, um die Richtlinienaktualisierung zu approximieren. Ein wichtiger Aspekt von PPO ist die Verwendung von abgeschnittenen Wahrscheinlichkeitsverhältnissen, um zu verhindern, dass die Richtlinienaktualisierungen zu groß werden und das Training destabilisieren. Das Ziel besteht darin, das Wahrscheinlichkeitsverhältnis der neuen Richtlinie im Vergleich zur alten Richtlinie zu maximieren, wird jedoch innerhalb eines bestimmten Bereichs abgeschnitten, um große Änderungen zu verhindern.


Vorteile:<br>
- Stabilität: PPO ist bekannt für seine Stabilität im Training von RL-Agenten. Es verwendet gekappte Wahrscheinlichkeitsverhältnisse, um zu verhindern, dass die Richtlinienaktualisierungen zu groß werden, was zu einem effizienten und konsistenten Lernen führt.
- On-Policy: PPO ist ein On-Policy-Algorithmus, was bedeutet, dass er die Richtlinie während des Trainings aktualisiert, indem er Daten von der aktuellen Richtlinie sammelt. Dies kann dazu beitragen, Probleme mit nicht-stationären Umgebungen zu vermeiden.
- Effiziente Nutzung von Daten: PPO verwendet Clipping und einen K-Lipschitz-Parameter, um die Lernrate zu begrenzen. Dadurch werden die Daten effizienter genutzt und das Lernen verbessert.

Nachteile:<br>
- Exploration vs. Exploitation: Obwohl PPO eine gewisse Exploration ermöglicht, kann es manchmal eine Balance zwischen Exploration und Exploitation schwierig sein. Die Balance kann durch die Wahl geeigneter Hyperparameter erreicht werden, aber das Tuning kann zeitaufwändig sein.
- Hyperparameter-Tuning: PPO hat mehrere Hyperparameter, die sorgfältig abgestimmt werden müssen, um optimale Ergebnisse zu erzielen.

In [29]:
# Modell erstellen und trainieren
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=20000)

Using cpu device
-----------------------------
| time/              |      |
|    fps             | 1097 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 846          |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0065650987 |
|    clip_fraction        | 0.0571       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.687       |
|    explained_variance   | -0.0127      |
|    learning_rate        | 0.0003       |
|    loss                 | 8.58         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00781     |
|    value_loss           | 54.9         |
------------------------------------------

In [30]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(484.4, 25.2)

### Erklärung der Werte

484.4:
- Dieser Wert ist der durchschnittliche Gesamtscore, den der Agent über die 10 Bewertungsepisoden erzielt hat. Der Gesamtscore ist die Gesamtbelohnung, die der Agent in einer Episode gesammelt hat. Ein höherer Durchschnittsscore zeigt an, dass der Agent in der Umgebung eine bessere Leistung erbracht hat. In diesem Fall erzielte der Agent einen durchschnittlichen Gesamtscore von etwa 484.4 über die Bewertungsepisoden.

25.2:
- Dieser Wert ist die Standardabweichung des Gesamtscores über die 10 Bewertungsepisoden. Die Standardabweichung misst die Streuung der Werte um den Durchschnitt. Eine niedrige Standardabweichung deutet darauf hin, dass die Ergebnisse in den Bewertungsepisoden näher beieinander liegen, was auf eine konsistente Leistung des Agenten hindeutet. In diesem Fall beträgt die Standardabweichung etwa 25.2.

## Eigene Implementation: Erweiterte oder tiefere Einblicke in die Ergebnisse

In [ ]:
import matplotlib.pyplot as plt
# Funktion zum Visualisieren des Agenten definieren
def visualize_agent(agent, env, episodes=5):
    for episode in range(episodes):
        state = env.reset()
        done = False
        total_reward = 0

        while not done:
            env.render()
            action, _ = agent.predict(state)  # Aktion vorhersagen
            state, reward, done, _ = env.step(action)
            total_reward += reward

        print(f"Episode {episode + 1}, Total Reward: {total_reward}")

    env.close()

# Visualisierung des Agenten über 5 Episoden
visualize_agent(model, env, episodes=5)